In [1]:
import MetaTrader5 as mt5
import pandas as pd
import numpy as np
from datetime import datetime
from arch import arch_model

def calculate_egarch(log_returns, omega=-0.1, beta=0.9, alpha=0.1, gamma=-0.1):
    n = len(log_returns)
    log_h = np.zeros(n)
    vol = np.zeros(n)
    log_h[0] = np.log(0.0001)
    vol[0] = np.sqrt(np.exp(log_h[0]))
    
    for t in range(1, n):
        prev_log_h = log_h[t-1]
        prev_h = np.exp(prev_log_h)
        prev_r = log_returns[t-1]
        prev_z = prev_r / np.sqrt(prev_h)
        log_h[t] = omega + beta * prev_log_h + alpha * (np.abs(prev_z) - np.sqrt(2/np.pi)) + gamma * prev_z
        vol[t] = np.sqrt(np.exp(log_h[t]))
    
    return vol
if not mt5.initialize():
    print("falha ao inicializar mt5")
    quit()

symbol = "DOL@"
start_date = datetime(2023, 1, 1)
end_date = datetime.now()
tickrate = mt5.TIMEFRAME_M5
rates = mt5.copy_rates_range(symbol, tickrate, start_date, end_date)
df = pd.DataFrame(rates)
df['time'] = pd.to_datetime(df['time'], unit='s')


df['log_returns'] = np.log(df['close'] / df['close'].shift(1))
df = df.dropna(subset=['log_returns'])
df['egarch_vol'] = calculate_egarch(df['log_returns'].values)
df['date'] = df['time'].dt.date
max_vol_per_day = df.loc[df.groupby('date')['egarch_vol'].idxmax()]
resultado = max_vol_per_day[['time', 'egarch_vol']].reset_index(drop=True)
resultado['hora'] = resultado['time'].dt.strftime('%H:%M')

print("\nresultados")
print(resultado[['time', 'hora', 'egarch_vol']].to_string(index=False))

mt5.shutdown()


resultados
               time  hora  egarch_vol
2023-01-02 18:10:00 18:10    0.407122
2023-01-03 09:35:00 09:35    0.407827
2023-01-04 13:15:00 13:15    0.407718
2023-01-05 09:20:00 09:20    0.407845
2023-01-06 10:50:00 10:50    0.408282
2023-01-09 13:55:00 13:55    0.407710
2023-01-10 11:10:00 11:10    0.407797
2023-01-11 10:25:00 10:25    0.407759
2023-01-12 10:50:00 10:50    0.408044
2023-01-13 10:45:00 10:45    0.407774
2023-01-16 13:00:00 13:00    0.407430
2023-01-17 10:10:00 10:10    0.407590
2023-01-18 10:50:00 10:50    0.407713
2023-01-19 10:45:00 10:45    0.407717
2023-01-20 13:05:00 13:05    0.407547
2023-01-23 10:10:00 10:10    0.407567
2023-01-24 10:10:00 10:10    0.407666
2023-01-25 09:10:00 09:10    0.407658
2023-01-26 12:25:00 12:25    0.407917
2023-01-27 10:35:00 10:35    0.407451
2023-01-30 10:50:00 10:50    0.407507
2023-01-31 11:05:00 11:05    0.407717
2023-02-01 16:45:00 16:45    0.407920
2023-02-02 10:35:00 10:35    0.408475
2023-02-03 09:50:00 09:50    0.407707


True